In [1]:
%matplotlib inline
#import cv2
#from matplotlib import pyplot as plt
import numpy as np
from sklearn.cluster import KMeans
from skimage import io
from skimage.color import rgb2gray, label2rgb, gray2rgb
from skimage.draw import circle_perimeter
from skimage.filters import threshold_otsu, threshold_local, median
from skimage.feature import canny
from skimage.measure import label, regionprops
from skimage.morphology import closing, square, disk, erosion
from skimage.segmentation import clear_border
from skimage.transform import hough_circle, hough_circle_peaks

In [2]:
im = cv2.imread('test_image.tif', cv2.IMREAD_GRAYSCALE)

thresh = threshold_otsu(im)
binary = im < thresh


NameError: name 'cv2' is not defined

In [ ]:
plt.imshow(im)

In [ ]:
im = cv2.imread('test_image.tif', cv2.IMREAD_GRAYSCALE)

thresh = threshold_local(im, block_size=41)
binary = im < thresh
im[binary] = 0
plt.imshow(im)

In [ ]:
io.imsave('test.tif', im)

In [ ]:
# Setup SimpleBlobDetector parameters.
params = cv2.SimpleBlobDetector_Params()
 
# Filter by Circularity
params.filterByCircularity = True
params.minCircularity = 0.1
 
# Filter by Inertia
params.filterByInertia = True
params.minInertiaRatio = 0.01

# Set up the detector with default parameters.
detector = cv2.SimpleBlobDetector_create(params)
 
# Detect blobs.
keypoints = detector.detect(im)
 
# Draw detected blobs as red circles.
# cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS ensures the size of the circle corresponds to the size of blob
im_with_keypoints = cv2.drawKeypoints(im, keypoints, np.array([]), (0,0,255), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
 

In [ ]:
f, ax = plt.subplots(figsize=(15, 10))
ax.imshow(im_with_keypoints)

In [ ]:
im = io.imread('1536_680nm.tif')

filtered = median(im, square(7))
thresh = threshold_local(filtered, block_size=101)
bw = closing(filtered > thresh, square(3))

cleared = clear_border(bw)
label_image = label(cleared)
image_label_overlay = label2rgb(label_image, image=im)

regions = regionprops(label_image)

#f, ax = plt.subplots(figsize=(15, 10))
#ax.imshow(image_label_overlay)

In [ ]:
%gui qt5
import napari


In [ ]:
viewer = napari.view_image(im)
viewer.add_labels(label_image)

eroded_labels = erosion(label_image, selem=disk(7))

viewer.add_labels(eroded_labels)



In [ ]:
eroded_regions = regionprops(eroded_labels)

areas = [r.area for r in eroded_regions]

area_thresh = threshold_otsu(np.asarray(areas))

plt.hist(areas)

print(area_thresh)

In [ ]:
len(eroded_regions)

In [ ]:
def calc_circularity(region):
    area = region.area
    perimeter = region.perimeter
    
    if perimeter > 0:
        circularity = 4 * area * np.pi / perimeter**2
    else:
        circularity = 0
    
    return circularity

In [ ]:
size_filtered = [r for r in eroded_regions if r.area > 100]
circularity = [calc_circularity(r) for r in size_filtered]

#circularity_filtered = [r for c, r in zip(circularity, size_filtered) if c > 0.6]
circularity_filtered = size_filtered

In [ ]:
areas = [r.area for r in circularity_filtered]

good_labels = [r.label for r in circularity_filtered]

unique_labels = np.unique(label_image)
in_good_labels = np.isin(unique_labels, good_labels)
labels_to_remove = unique_labels[np.logical_not(in_good_labels)]

good_label_im = label_image
for l in labels_to_remove:
    good_label_im[good_label_im == l] = 0

In [ ]:
image_label_overlay = label2rgb(good_label_im, image=im, bg_label=0, image_alpha=0.8)

f, ax = plt.subplots(figsize=(15, 10))
ax.imshow(image_label_overlay)

In [ ]:
coords = np.array([r.centroid for r in circularity_filtered])

In [ ]:
coords[:, 1].max()

In [ ]:
binary_mask = good_label_im > 0

plt.imshow(binary_mask)

In [ ]:
napari.view_labels(binary_mask)

In [ ]:
image_label_overlay = label2rgb(good_label_im, image=im, bg_color=[0, 0, 0], image_alpha=1)

f, ax = plt.subplots(figsize=(15, 10))
ax.imshow(image_label_overlay)

In [ ]:
unique_labels = np.unique(label_image)

In [ ]:
src = im

gray = cv2.medianBlur(im, 7)

circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, 1, 20,
                               param1=100, param2=30,
                               minRadius=1, maxRadius=20)

if circles is not None:
    circles = np.uint16(np.around(circles))
    for i in circles[0, :]:
        center = (i[0], i[1])
        # circle center
        cv2.circle(src, center, 1, (0, 100, 100), 3)
        # circle outline
        radius = i[2]
        cv2.circle(src, center, radius, (255, 0, 255), 3)

In [ ]:
def make_grid(n_row, n_col, jitter):
    GRID_SPACING = 10
    
    x = np.arange(0, GRID_SPACING*n_col, GRID_SPACING)
    y = np.arange(0, GRID_SPACING*n_row, GRID_SPACING)
                  
    grid_x, grid_y = np.meshgrid(x, y)
    
    x = grid_x.ravel()
    y = grid_y.ravel()

    jitter_x = jitter * (np.random.random(size=x.shape) - 0.5)
    jitter_y = jitter * (np.random.random(size=y.shape) - 0.5)
    
    coords_x = x + jitter_x
    coords_y = y + jitter_y
    
    return coords_x, coords_y

In [ ]:
x, y = make_grid(5, 5, 2)
coords = np.array([[x, y] for x, y in zip(x, y)])

In [ ]:
plt.plot(x, y, 'ok')

In [ ]:
def find_indices(coords):

    kmeans = KMeans(n_clusters=5, random_state=0).fit(coords.reshape(-1, 1))
    labels = kmeans.labels_

    indices = np.argsort(kmeans.cluster_centers_.ravel()).argsort()
    
    return labels, indices

In [ ]:
col_labels, col_indices = find_indices(x)
row_labels, row_indices = find_indices(y)

In [ ]:
col_indices

In [ ]:
plt.scatter(x, y, c=col_labels)

In [ ]:
plt.scatter(x, y, c=row_labels)

In [ ]:
well_coords = [[row_indices[row], col_indices[col]] for row, col in zip(row_labels, col_labels)]

In [ ]:
well_coords